# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [2]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [3]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [4]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [5]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [ ]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
Here is another example, imagine that the list of link is "/goes/here/sustainability" from the website https://full.url and https://another.full.url/news", the result should be:
{
    "links": [
        {"type": "Sustainability page", "url": "https://full.url/goes/here/sustainability"},
        {"type": "news page", "url": "https://another.full.url/news"}
    ]
}
"""

In [7]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}



In [8]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [9]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddo

In [10]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [11]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/baidu/ERNIE-4.5-21B-A3B-Thinking',
 '/tencent/HunyuanImage-2.1',
 '/google/embeddinggemma-300m',
 '/Qwen/Qwen3-Next-80B-A3B-Instruct',
 '/openbmb/MiniCPM4.1-8B',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/zerogpu-aoti/wan2-2-fp8da-aoti-faster',
 '/spaces/multimodalart/wan-2-2-first-last-frame',
 '/spaces/Qwen/Qwen3-ASR-Demo',
 '/spaces/tencent/HunyuanImage-2.1',
 '/spaces',
 '/datasets/HuggingFaceFW/finepdfs',
 '/datasets/HuggingFaceM4/FineVision',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/Pageshift-Entertainment/LongPage',
 '/datasets/jupyter-agent/jupyter-agent-dataset',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',

In [12]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/home'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'company page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'community page', 'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [13]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [14]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
baidu/ERNIE-4.5-21B-A3B-Thinking
Updated
1 day ago
•
62.2k
•
559
tencent/HunyuanImage-2.1
Updated
about 7 hours ago
•
495
•
542
google/embeddinggemma-300m
Updated
2 days ago
•
124k
•
698
Qwen/Qwen3-Next-80B-A3B-Instruct
Updated
about 17 ho

In [36]:
#system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
#and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
#Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information. Each title should have an icon. The last section should always start with Conclusion: and then add a joke in the title"


In [37]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [38]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nbaidu/ERNIE-4.5-21B-A3B-Thinking\nUpdated\n1 day ago\n•\n62.2k\n•\n559\ntencent/HunyuanImage-2.1\nUpdated\nabout 7 hours ago\n•\n495\n•\n542\ngoogle/embeddinggemma-300m\nUpdated\n2 days ago\n•\n124k\n•\n700\nQwen/Qwen3-Next-80B-A3B-Instruct\nUpdated\nabout 18 hours ago\n•\n19.6k\n•\n381\nopenbmb/MiniCPM4.1-8B\nUpdated\nabout 5 hours ago\n•\n2.13k\n•\n291\nBrowse 1M+ models\nSpaces\nRunning

In [42]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    result = result.replace("```","").replace("markdown", "")
    return Markdown(result)

In [43]:
res = create_brochure("HuggingFace", "https://huggingface.co")
display(res)

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}



# 🌟 Hugging Face: The AI Community Building the Future!

Welcome to Hugging Face — where our virtual hugs are definitely not AI-generated! 🤗 We're the friendly neighborhood platform serving up AI goodness to enthusiasts, enterprises, and everyone in-between. Think of us as the Starbucks of AI; we provide a cozy space for ML models, datasets, and applications to brew together!

---

## 🤖 Let's Talk Models

At Hugging Face, we have **over 1 million models** just waiting for you to explore! From the brainy baidu/ERNIE-4.5-21B-A3B-Thinking (which is probably more intelligent than your ex) to the artistic tencent/HunyuanImage-2.1 which generates glorious images like a modern Picasso. 🖼️ Come for the models, stay to tell your friends you’re AI-savvy!

---

## 📊 The A+ in Datasets

Ever search for a dataset and feel like you should’ve just Googled ‘cats’? Forget about it! We have over **250,000 datasets** perfect for your ML needs. Want to predict the weather? We have that. Want to analyze dad jokes? Well… we don't have that yet, but we are working on it! 😜 

---

## 🚀 Spaces: The Place to Be!

Our Spaces are like an interactive playground but for AI. With **13,000+ running applications**, there's something for everyone! You can generate videos from images, convert audio to text, and even amuse your friends with deep fakes! 🤫 (We won’t tell if you don’t!)

---

## 💼 Join the Hugging Family!

If you’re looking for a career, Hugging Face is the place to be! Our company culture is as vibrant as a rainbow unicorn — think open-source collaboration, a friendly community, and a coffee pot that never runs dry. 🚀🌈 Recent job openings include ML Engineers, Data Scientists, and Chief Unicorn Wranglers (it's a tough job but someone has to do it!). 

---

## 📈 Who Loves Us?

More than **50,000 organizations** are already using Hugging Face, including mega-players like Google, Microsoft, and Amazon. And yes, we plan to take over the world... one model at a time! 🌏

---

## 🎉 Join the Fun!

So why not come join our lovable community? Whether you’re a budding coder, an AI guru, or just here for the memes, we have a little something for everyone.

### Conclusion: 
Why did the computer show up at work late? 
**Because it had a hard drive!** 😂



# Translate to spanish

In [44]:
def format_message(prompt):
    messages = [
        {"role": "system", "content": "You are an expert translator that translates brochures from english to spanish. You always make sure to keep the same tone and structure as the original brochure. Always answer in Markdown format."},
        {"role": "user", "content": f"Here is a brochure in english, please translate it to spanish while keeping the same tone and structure:\n\n{prompt}"},
    ]
    return messages

In [50]:
def translate_brochure(prompt):
    messages = format_message(prompt)
    print(messages)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=messages,
        stream=False
    )
    return Markdown(response.choices[0].message.content)

In [52]:
display(translate_brochure(res.data))

[{'role': 'system', 'content': 'You are an expert translator that translates brochures from english to spanish. You always make sure to keep the same tone and structure as the original brochure. Always answer in Markdown format.'}, {'role': 'user', 'content': "Here is a brochure in english, please translate it to spanish while keeping the same tone and structure:\n\n\n# 🌟 Hugging Face: The AI Community Building the Future!\n\nWelcome to Hugging Face — where our virtual hugs are definitely not AI-generated! 🤗 We're the friendly neighborhood platform serving up AI goodness to enthusiasts, enterprises, and everyone in-between. Think of us as the Starbucks of AI; we provide a cozy space for ML models, datasets, and applications to brew together!\n\n---\n\n## 🤖 Let's Talk Models\n\nAt Hugging Face, we have **over 1 million models** just waiting for you to explore! From the brainy baidu/ERNIE-4.5-21B-A3B-Thinking (which is probably more intelligent than your ex) to the artistic tencent/Hunyu

# 🌟 Hugging Face: ¡La Comunidad de IA Construyendo el Futuro!

¡Bienvenido a Hugging Face — donde nuestros abrazos virtuales definitivamente no son generados por IA! 🤗 Somos la plataforma amigable del vecindario que ofrece bondades de IA a entusiastas, empresas y a todo el que esté en medio. ¡Piénsanos como el Starbucks de la IA; proporcionamos un espacio acogedor para que modelos de ML, conjuntos de datos y aplicaciones se mezclen!

---

## 🤖 Hablemos de Modelos

En Hugging Face, tenemos **más de 1 millón de modelos** esperando a que los explores. Desde el inteligente baidu/ERNIE-4.5-21B-A3B-Thinking (que probablemente es más inteligente que tu ex) hasta el artístico tencent/HunyuanImage-2.1 que genera imágenes gloriosas como un Picasso moderno. 🖼️ ¡Ven por los modelos, quédate para decirles a tus amigos que eres un experto en IA!

---

## 📊 La A+ en Conjuntos de Datos

¿Alguna vez has buscado un conjunto de datos y sentiste que deberías haber buscado ‘gatos’? ¡Olvídalo! Tenemos más de **250,000 conjuntos de datos** perfectos para tus necesidades de ML. ¿Quieres predecir el clima? Lo tenemos. ¿Quieres analizar chistes de papá? Bueno... aún no lo tenemos, ¡pero estamos trabajando en eso! 😜 

---

## 🚀 Espacios: ¡El Lugar para Estar!

Nuestros Espacios son como un patio de recreo interactivo, pero para IA. Con **más de 13,000 aplicaciones en funcionamiento**, ¡hay algo para todos! Puedes generar videos a partir de imágenes, convertir audio a texto, ¡e incluso divertir a tus amigos con deep fakes! 🤫 (¡No se lo diremos si tú no lo haces!)

---

## 💼 ¡Únete a la Familia Hugging!

Si estás buscando una carrera, ¡Hugging Face es el lugar para estar! Nuestra cultura empresarial es tan vibrante como un unicornio arcoíris — piensa en colaboración de código abierto, una comunidad amigable y una cafetera que nunca se queda vacía. 🚀🌈 Las ofertas de trabajo recientes incluyen Ingenieros de ML, Científicos de Datos y Jefes de Unicornio (es un trabajo duro, pero alguien tiene que hacerlo!).

---

## 📈 ¿Quiénes nos Aman?

Más de **50,000 organizaciones** ya están usando Hugging Face, incluyendo mega-jugadores como Google, Microsoft y Amazon. Y sí, planeamos conquistar el mundo... ¡modelo por modelo! 🌏

---

## 🎉 ¡Únete a la Diversión!

Entonces, ¿por qué no venir a unirte a nuestra adorable comunidad? Ya seas un programador en crecimiento, un gurú de IA, o solo estés aquí por los memes, tenemos un pequeño algo para todos.

### Conclusión:
¿Por qué llegó tarde la computadora al trabajo? 
**¡Porque tuvo un disco duro!** 😂

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [23]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [24]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'contact page', 'url': 'https://huggingface.co/contact'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}


# Welcome to the Hugging Face Universe! 🌌

### The AI Community Building the Future (One Hug at a Time)

---

### What We Do
At Hugging Face, we’re not just building AI models; we’re creating a **hugging utopia** where happy nerds can collaboratively explore the realms of machine learning. Dive into our treasure trove of **1M+ models**, **250k+ datasets**, and **400k+ applications**. That’s right, if Elon Musk can send a rocket to Mars, we can definitely send your AI aspirations soaring!

---

### Our Models Are Like Potato Chips
You can’t just have one! From the **baidu/ERNIE-4.5-21B-A3B-Thinking** (which is clearly an overthinker) to **google/embeddinggemma-300m** (because who doesn’t want a little bit of everything?), our models are updated continuously and are absolutely packed with potential! 

#### Trending This Week: 
- **baidu/ERNIE-4.5-21B-A3B-Thinking** (Updated yesterday - 62.2k minds read it and loved it!)
- **tencent/HunyuanImage-2.1** (Updated 7 hours ago - 495 likes and counting!)
  
You’re just a click away from your new favorite algorithm!

---

### The Workplace Culture: Hug-ly Inclusive 🤗
Our team is a whole bunch of talents (not just the robot kind) fuelled by solidarity, laughter, and coffee that would wake the sleepiest of bots! We're all about **collaboration**! Employees enjoy access to **enterprise-grade support**, reassuring security (no “skynet” drama here!), and the freedom to explore the weird and wonderful realms of ML. 

---

### Customers: Who Might Need a Hug?
More than **50,000 organizations** have joined our family, including the cool kids from Meta, Amazon, and Microsoft! After all, who else can you call when your AI needs a little TLC (tender, loving computation)? Here, your insights and needs receive the most gentle of handling—like a newborn AI puppy!

---

### Join the Hugging Face Family: Careers!
We’re looking for individuals who don't just want a job, but rather want to be part of a quirky, energetic, innovative team that’s reshaping the future! At Hugging Face, you won’t just climb the corporate ladder—you’ll be diving into the deep end of the ML pool! 🌊

- **Open Positions**: If you can code, compile datasets, or chat with bots like a fine digital linguist, we want to hear from you!
- **Benefits**: Competitive salaries, an endless supply of virtual hugs, and networking with some of the best brains in the business. Plus, did we mention the unlimited coffee?

---

### Join Us!
So, whether you're an AI enthusiast, a budding developer, a company looking for AI solutions, or a caffeine-fueled coder, come find your **place at Hugging Face**! It’s time to embark on your journey into the future. Because remember, in the world of AI, it’s always better with a hug.

![Hugging Face Logo](https://huggingface.co/assets/huggingface-logo.svg)

---

**Let's Build the Future Together!** 

**[Explore Now](https://huggingface.co/) | [Join the Community](https://huggingface.co/community)**

---

*Disclaimer: Results of hugging may vary depending on AI's transformative power. Expect strong, innovative outcomes and possibly a few dad jokes!*

In [55]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [ ]:
import gradio as gr

gr.Interface(
    fn=stream_brochure,
    inputs=[
        gr.Textbox(label="Company Name"),
        gr.Textbox(label="Company URL")
    ],
    outputs=[
        gr.Markdown(label="Brochure")
    ],
    flagging_mode="never"

).launch(share=True)

* Running on local URL:  http://127.0.0.1:7871
Found links: {'links': [{'type': 'about page', 'url': 'https://www.squareflow.be/about-us'}, {'type': 'careers page', 'url': 'https://www.squareflow.be/join-the-team'}, {'type': 'contact page', 'url': 'https://www.squareflow.be/contactus'}]}
* Running on public URL: https://a2bd063760e4cbb4ba.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Found links: {'links': [{'type': 'about page', 'url': 'https://www.squareflow.be/about-us'}, {'type': 'careers page', 'url': 'https://www.squareflow.be/join-the-team'}, {'type': 'contact page', 'url': 'https://www.squareflow.be/contactus'}]}


Found links: {'links': [{'type': 'about page', 'url': 'https://www.squareflow.be/about-us'}, {'type': 'careers page', 'url': 'https://www.squareflow.be/join-the-team'}, {'type': 'contact page', 'url': 'https://www.squareflow.be/contactus'}]}


Traceback (most recent call last):
  File "/opt/miniconda3/envs/llms/lib/python3.11/site-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/envs/llms/lib/python3.11/site-packages/urllib3/util/connection.py", line 60, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/envs/llms/lib/python3.11/socket.py", line 974, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
socket.gaierror: [Errno 8] nodename nor servname provided, or not known

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/opt/miniconda3/envs/llms/lib/python3.11/site-packages/urllib3/connectionpool.p

Found links: {'links': [{'type': 'about page', 'url': 'https://erebor.odoo.com/about-us'}, {'type': 'contact page', 'url': 'https://erebor.odoo.com/contactus'}, {'type': 'services page', 'url': 'https://erebor.odoo.com/our-services'}]}


Found links: {'links': [{'type': 'about page', 'url': 'https://www.gsk.com/en-gb/company/'}, {'type': 'company culture', 'url': 'https://www.gsk.com/en-gb/company/purpose-strategy-and-culture/'}, {'type': 'careers page', 'url': 'https://www.gsk.com/en-gb/careers/'}, {'type': 'featured careers', 'url': 'https://www.gsk.com/en-gb/careers/featured-careers/'}, {'type': 'company overview', 'url': 'https://www.gsk.com/en-gb/company/gsk-at-a-glance/'}, {'type': 'company history', 'url': 'https://www.gsk.com/en-gb/company/history-and-heritage/'}, {'type': 'leadership team', 'url': 'https://www.gsk.com/en-gb/company/board-of-directors-and-leadership-team/'}, {'type': 'innovation overview', 'url': 'https://www.gsk.com/en-gb/innovation/'}, {'type': 'responsibility page', 'url': 'https://www.gsk.com/en-gb/responsibility/'}, {'type': 'investor relations', 'url': 'https://www.gsk.com/en-gb/investors/'}, {'type': 'products page', 'url': 'https://www.gsk.com/en-gb/products/'}]}


In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>